# Instalación de la librerías necesarias

In [1]:
!pip install langchain==0.1.20 -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install pypdf -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.3 MB/s eta 0:00:00


# Cargamos información de un sitio web

In [3]:
!pip install beautifulsoup4

In [30]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.infobae.com/")
datos_web = loader.load()

In [31]:
print(datos_web)

[Document(page_content='Infobae América - Infobae\n\n19 Jun, 2024BuscarArgentinaColombiaEspañaMéxicoPerúMundoVenezuelaThe economistThe Washington PostRealezaOpiniónCopa AméricaNuevoEurocopaNuevoÚltimas NoticiasEntretenimientoDeportesTendenciasQué puedo verTecnoCulturaMalditos NerdsViralesSaludPodcastsJuegosNewsletters;BienvenidoPor favor, ingresa a tu cuenta.REGISTRARMEINICIAR SESIÓNCopa AméricaEurocopaInvasión de Rusia a UcraniaCrisis en VenezuelaGuerra en Medio OrienteInteligencia artificialNewslettersCómo funciona la alianza entre el Cártel de Sinaloa y la banca clandestina de China para lavar dineroEl esquema detectado por el Departamento de Justicia de EEUU incluye a ciudadanos chinos que buscan conseguir dólares estadounidensesEl movimiento que planea el príncipe Andrés para evitar abandonar Royal Lodge como le exige Carlos IIIPor Anabel Gómez SalvadorPor Anabel Gómez SalvadorTras la victoria de Alemania, Suiza y Escocia empataron en la segunda fecha de la Eurocopa\n\n\n¿Qué tan 

# Dividimos la información de la web en partes

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [34]:
datos_web_chunks = text_splitter.split_documents(datos_web)

In [35]:
print(f"Ahora tenemos {len(datos_web_chunks)} partes.")

Ahora tenemos 6 partes.


# Inicializamos el modelo de embeddings que vamos a utilizar para convertir las partes de texto en vectores numéricos (embeddings)

In [36]:
!pip install langchain-openai==0.1.3 -Uq

In [37]:
from langchain_openai import OpenAIEmbeddings

In [38]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [39]:
embeddings_model = OpenAIEmbeddings(api_key=openai_api_key)

# Cargamos los embeddings en una base de datos vectorial

In [41]:
!pip install faiss-cpu -Uq

In [43]:
from langchain.vectorstores import FAISS

In [44]:
stored_embeddings = FAISS.from_documents(datos_web_chunks, embeddings_model)

# Creamos una Cadena de Preguntas y Respuetas con Recuperación

In [45]:
from langchain_openai import OpenAI

In [46]:
llm = OpenAI(api_key=openai_api_key)

In [47]:
from langchain.chains import RetrievalQA

In [48]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

# Probamos nuestra aplicación de Preguntas y Respuestas relacionadas al documento de interés

In [55]:
pregunta1 = """
De qué trata la web?
"""

In [57]:
respuesta1 = QA_chain.invoke(pregunta1)

In [60]:
print(respuesta1['result'])


La web trata principalmente sobre noticias y entretenimiento, con secciones sobre deportes, tecnología y tendencias. También ofrece juegos y newsletters para sus lectores.


In [61]:
pregunta2 = """
Qué noticias de deportes hay??
"""

In [62]:
respuesta2 = QA_chain.invoke(pregunta2)

In [63]:
print(respuesta2['result'])


1. "El futbolista de 21 años que dirige cuatro empresas que generan millones de euros al año: “Es un hobbie”"
2. "La primera imagen de Kylian Mbappé después de fracturarse la nariz en la selección de Francia"
3. "Boca Juniors se enfrentará a Almirante Brown por el pase a los octavos de final de la Copa Argentina, en vivo: hora, TV y formaciones"
4. "Duro golpe para los Pumas 7 de cara a los Juegos Olímpicos: la sanción que recibió una de sus figuras"
5. "La palabra de Scaloni en la previa del debut de la Selección en la Copa América: una sorpresa confirmada y el exigente mensaje al plantel"
6. "Las palabras de Federer sobre la rivalidad entre Alcaraz y Siner: 'Son muy fuertes y se respetan mucho...'"
7. "Liverpool presentó al 23° entrenador de su historia: Arne Slot reemplazará a Jürgen Klopp"
8. "Manchester City presentó nuevas camisetas con la letra


In [64]:
pregunta3 = """
Qué noticias de tecnología hay?
"""

In [65]:
respuesta3 = QA_chain.invoke(pregunta3)

In [66]:
print(respuesta3['result'])

 
Algunas noticias de tecnología actuales incluyen el aumento en el consumo problemático de marihuana entre los adultos mayores en Estados Unidos, la alianza entre el Cártel de Sinaloa y la banca clandestina de China para lavar dinero, y el valor de diferentes criptomonedas como Bitcoin y Tether. También hay noticias sobre nuevos juegos y películas de entretenimiento, como la adaptación de Dragon Quest III a HD y la nueva película de terror de Kit Harington.
